In [37]:
import pandas as pd
import ast
from urllib.parse import urlparse, parse_qs

In [38]:
""" Replace YT URLs with Vimeo URLS """
# Get vimeo
yt_vimeo_pairs = pd.read_excel("yt_vimeo_pairs.xlsx")
yt_vimeo_pairs["Vimeo"] = yt_vimeo_pairs["Vimeo"].apply(lambda x: x.replace("/manage/videos", ""))
yt_vimeo_pairs.head()

# Get current lib
lib = pd.read_excel("Backup-28-02-2023.xlsx")

# Extract and clean yt links in lib
yt_vimeo_pairs.head()

def parse_url(metadata: str):
    # Get url from metadata dict
    return ast.literal_eval(metadata)["id"]

def parse_id(url: str):
    # parse id from youtube url
    parse_result = urlparse(url)
    query_params = parse_qs(parse_result.query)
    return query_params["v"][0]

# Add video ids as primary keys
lib["Video_ID"] = lib["Metadata"].apply(lambda metadata: parse_id(parse_url(metadata)))
yt_vimeo_pairs["Video_ID"] = yt_vimeo_pairs["YouTube"].apply(parse_id)

# deduplicate pairs
num_dumplicates = sum(yt_vimeo_pairs.duplicated(subset=["Video_ID"]))
if num_dumplicates > 0:
    print(f"Found {num_dumplicates} duplicate yt videos. Currently {len(yt_vimeo_pairs)} pairs.")
    print("Deduplicating ...")
    yt_vimeo_pairs.drop_duplicates(subset=["Video_ID"], inplace=True, ignore_index=True)
    print(f"{len(yt_vimeo_pairs)} pairs left after deduplication")

# Join on Video IDs
new_lib = pd.merge(lib, yt_vimeo_pairs, how="left", on="Video_ID")
assert len(new_lib) == len(lib)

# Validate new lib
print("Following videos do not have a vimeo link:")
missing_links = new_lib[new_lib["Vimeo"].isna()].drop_duplicates(subset=["Video_ID"]).reset_index()
display(missing_links)

# Save them for reference
missing_links.to_excel("missing_links.xlsx")

# Drop videos with missing links
print("Dropping videos without link")
new_lib.dropna(subset="Vimeo", inplace=True)

# Update metadata column
def replace_link_in_metadata(metadata_dict, vimeo_link):
    metadata = ast.literal_eval(metadata_dict)
    metadata["id"] = vimeo_link
    metadata["video_id"] = parse_id(parse_url(metadata_dict))
    return str(metadata)

new_lib["Metadata"] = new_lib.apply(lambda x: replace_link_in_metadata(x["Metadata"], x["Vimeo"]), axis=1)
display(new_lib)

# Remove unnecessary columns
new_lib = new_lib[["Name", "Content", "Tags", "Metadata", "Context", "Color", "Comment"]]
display(new_lib)

# Dump new library
new_lib.to_excel("vimeo_lib.xlsx")

                         Playlist  \
0  gebrochen rationale Funktionen   
1                             NaN   
2                             NaN   
3                             NaN   
4                             NaN   

                                             YouTube  \
0  https://www.youtube.com/watch?v=5yNQTUdpX2s&li...   
1  https://www.youtube.com/watch?v=WN47VxNcNp8&li...   
2  https://www.youtube.com/watch?v=Ooid7jrf9L4&li...   
3  https://www.youtube.com/watch?v=ZGeRu1-VjJ8&li...   
4  https://www.youtube.com/watch?v=46vLgsLDzzM&li...   

                         Vimeo  
0  https://vimeo.com/799060557  
1  https://vimeo.com/799060602  
2  https://vimeo.com/799060639  
3  https://vimeo.com/799060675  
4  https://vimeo.com/799060713  


C:\Users\TomKaminski\Documents\AIEDN\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Found 92 duplicate yt videos. Currently 643 pairs.
Deduplicating ...
551 pairs left after deduplication
Following videos do not have a vimeo link:


,index,Name,Content,Tags,Metadata,Context,Color,Comment,Video_ID,Playlist,YouTube,Vimeo
0,3,"Funktionen, E(x) Versionen, Wirtschaftsanwendu...","Funktionen, E(x) Versionen, Wirtschaftsanwendu...","Polynomfunktionen, base, Ganzrationale Funktio...","{'start': 11, 'end': 65, 'id': 'https://www.yo...",NaN,NaN,NaN,-TZPuJaJV1A,NaN,NaN,NaN
1,9,"Grenzverhalten, Grenzwert, Was ist Was Teil 1,...","Grenzverhalten, Grenzwert, Was ist Was Teil 1,...","Grenzverhalten, 11, Unendlich..., Grenzwert, C...","{'start': 0, 'end': 22, 'id': 'https://www.you...",NaN,NaN,NaN,0c82055Tvwg,NaN,NaN,NaN
2,49,Regel von l'Hospital Beispiel | Mathe by Danie...,Regel von l'Hospital Beispiel | Mathe by Danie...,"11, Grenzwert, Unendlich..., Limes, Grenzverha...","{'start': 0, 'end': 34, 'id': 'https://www.you...",NaN,NaN,NaN,58W1oArE8p8,NaN,NaN,NaN
3,71,"Grenzwert, Grenzverhalten, Limes Einstieg | Ma...","Grenzwert, Grenzverhalten, Limes Einstieg | Ma...","CONTROL, Grenzverhalten, Unendlich..., Grenzwe...","{'start': 0, 'end': 70, 'id': 'https://www.you...",NaN,NaN,NaN,8Snag_AgNyU,NaN,NaN,NaN
4,87,"Grenzwert an einer Stelle anschaulich, linksse...","Grenzwert an einer Stelle anschaulich, linksse...","Grenzverhalten, Unendlich..., Grenzwert, 11, C...","{'start': 0, 'end': 40, 'id': 'https://www.you...",NaN,NaN,NaN,AFiOJfQDs6E,NaN,NaN,NaN
5,89,"Differenzierbarkeit, Stetigkeit, Folgerungen, ...","Differenzierbarkeit, Stetigkeit, Folgerungen, ...","Unendlich..., Grenzverhalten, Grenzwert, CONTR...","{'start': 310, 'end': 349, 'id': 'https://www....",NaN,NaN,NaN,AoA2T3dPuwk,NaN,NaN,NaN
6,101,"Auf Grenzwert untersuchen, cos(n*Pi), Beispiel...","Auf Grenzwert untersuchen, cos(n*Pi), Beispiel...","11, Grenzwert, Limes, Unendlich..., Grenzverha...","{'start': 0, 'end': 47, 'id': 'https://www.you...",NaN,NaN,NaN,COjis7G-aQY,NaN,NaN,NaN
7,109,"Folgen, Beschränktheit, Monotonie, Aufgabenbei...","Folgen, Beschränktheit, Monotonie, Aufgabenbei...","Unendlich..., Grenzwert, Limes, CONTROL, Grenz...","{'start': 0, 'end': 41, 'id': 'https://www.you...",NaN,NaN,NaN,D2dZ-z2e6K4,NaN,NaN,NaN
8,117,"Grenzverhalten, Grenzwert, Was ist Was Teil 2,...","Grenzverhalten, Grenzwert, Was ist Was Teil 2,...","CONTROL, Grenzverhalten, Grenzwert, 11, Limes,...","{'start': 0, 'end': 26, 'id': 'https://www.you...",NaN,NaN,NaN,DZu5M0g5IbQ,NaN,NaN,NaN
9,123,"0,999...ist gleich 1 (Teil 1), Folgen, Reihen,...","0,999...ist gleich 1 (Teil 1), Folgen, Reihen,...","Unendlich..., Grenzverhalten, CONTROL, Limes, ...","{'start': 0, 'end': 33, 'id': 'https://www.you...",NaN,NaN,NaN,E5pl7b1OChE,NaN,NaN,NaN


Dropping videos without link


,Name,Content,Tags,Metadata,Context,Color,Comment,Video_ID,Playlist,YouTube,Vimeo
0,Graph zur Ableitung und Stammfunktion | Mathe ...,Graph zur Ableitung und Stammfunktion | Mathe ...,"11, Ableitung, Differenzieren, Ableiten, Diffe...","{'start': 0, 'end': 46, 'id': 'https://vimeo.c...",NaN,NaN,NaN,-1ecuhQJwNU,NaN,https://www.youtube.com/watch?v=-1ecuhQJwNU&li...,https://vimeo.com/799044688
1,"Tangente, Tangentengleichung aufstellen mittel...","Tangente, Tangentengleichung aufstellen mittel...","CONTROL, Kurvendiskussion, 11, Bausteine allge...","{'start': 0, 'end': 43, 'id': 'https://vimeo.c...",NaN,NaN,NaN,-AyUqLeTtrE,NaN,https://www.youtube.com/watch?v=-AyUqLeTtrE&li...,https://vimeo.com/799069612
2,Ableitung Implizite Gleichung | Mathe by Danie...,Ableitung Implizite Gleichung | Mathe by Danie...,"Differentiation, Differenziation, CONTROL, Dif...","{'start': 0, 'end': 52, 'id': 'https://vimeo.c...",NaN,NaN,NaN,-PqSWya_6Vw,NaN,https://www.youtube.com/watch?v=-PqSWya_6Vw&li...,https://vimeo.com/799043322
4,"Monotonie, Krümmung bei Funktionen, Übersicht ...","Monotonie, Krümmung bei Funktionen, Übersicht ...","CONTROL, Polynomfunktionen, base, Analysis, Ga...","{'start': 1, 'end': 23, 'id': 'https://vimeo.c...",NaN,NaN,NaN,-fk6qn0c_Jk,NaN,https://www.youtube.com/watch?v=-fk6qn0c_Jk&li...,https://vimeo.com/799056242
5,"Kettenregel zum Ableiten, Basics | Mathe by Da...","Kettenregel zum Ableiten, Basics | Mathe by Da...","Ableitung, Differenzieren, Differenziation, CO...","{'start': 0, 'end': 43, 'id': 'https://vimeo.c...",NaN,NaN,NaN,-nG1PpIZ2jo,NaN,https://www.youtube.com/watch?v=-nG1PpIZ2jo&li...,https://vimeo.com/799045917
...,...,...,...,...,...,...,...,...,...,...,...
17877,"Punkt, fehlender Wert, Punktprobe mit gegebene...","Ihr könnt es also jetzt auch noch zuordnen, ob...","Funktionen Übersicht, Funktionstypen, SENTENCE...","{'start': 340, 'end': 371, 'id': 'https://vime...",NaN,NaN,NaN,NtbtAwGJpvs,NaN,https://www.youtube.com/watch?v=NtbtAwGJpvs&li...,https://vimeo.com/799051404
17878,Schnittpunkt(e) bei Funktionen BASICS | Mathe ...,"Ja, einmal zum Thema Schnittpunkte bei Funktio...","base, Funktionstypen, SENTENCE_LEVEL, Funktion...","{'start': 1, 'end': 32, 'id': 'https://vimeo.c...",NaN,NaN,NaN,bwUTp00LRwU,NaN,https://www.youtube.com/watch?v=bwUTp00LRwU&li...,https://vimeo.com/799051463
17879,Punkt und Koordinaten | Mathe by Daniel Jung_2,"Also wir haben eine x-Achse, eine y-Achse.","Funktionsarten, base, SENTENCE_LEVEL, Funktion...","{'start': 8, 'end': 34, 'id': 'https://vimeo.c...",NaN,NaN,NaN,ABTzp0xCSlk,NaN,https://www.youtube.com/watch?v=ABTzp0xCSlk&li...,https://vimeo.com/799073415
17880,Schnittpunkt(e) bei Funktionen BASICS | Mathe ...,"Nochmal, was heißt das überhaupt ganz genau?","SENTENCE_LEVEL, base, Funktionen Übersicht, Fu...","{'start': 6, 'end': 37, 'id': 'https://vimeo.c...",NaN,NaN,NaN,bwUTp00LRwU,NaN,https://www.youtube.com/watch?v=bwUTp00LRwU&li...,https://vimeo.com/799051463


,Name,Content,Tags,Metadata,Context,Color,Comment
0,Graph zur Ableitung und Stammfunktion | Mathe ...,Graph zur Ableitung und Stammfunktion | Mathe ...,"11, Ableitung, Differenzieren, Ableiten, Diffe...","{'start': 0, 'end': 46, 'id': 'https://vimeo.c...",NaN,NaN,NaN
1,"Tangente, Tangentengleichung aufstellen mittel...","Tangente, Tangentengleichung aufstellen mittel...","CONTROL, Kurvendiskussion, 11, Bausteine allge...","{'start': 0, 'end': 43, 'id': 'https://vimeo.c...",NaN,NaN,NaN
2,Ableitung Implizite Gleichung | Mathe by Danie...,Ableitung Implizite Gleichung | Mathe by Danie...,"Differentiation, Differenziation, CONTROL, Dif...","{'start': 0, 'end': 52, 'id': 'https://vimeo.c...",NaN,NaN,NaN
4,"Monotonie, Krümmung bei Funktionen, Übersicht ...","Monotonie, Krümmung bei Funktionen, Übersicht ...","CONTROL, Polynomfunktionen, base, Analysis, Ga...","{'start': 1, 'end': 23, 'id': 'https://vimeo.c...",NaN,NaN,NaN
5,"Kettenregel zum Ableiten, Basics | Mathe by Da...","Kettenregel zum Ableiten, Basics | Mathe by Da...","Ableitung, Differenzieren, Differenziation, CO...","{'start': 0, 'end': 43, 'id': 'https://vimeo.c...",NaN,NaN,NaN
...,...,...,...,...,...,...,...
17877,"Punkt, fehlender Wert, Punktprobe mit gegebene...","Ihr könnt es also jetzt auch noch zuordnen, ob...","Funktionen Übersicht, Funktionstypen, SENTENCE...","{'start': 340, 'end': 371, 'id': 'https://vime...",NaN,NaN,NaN
17878,Schnittpunkt(e) bei Funktionen BASICS | Mathe ...,"Ja, einmal zum Thema Schnittpunkte bei Funktio...","base, Funktionstypen, SENTENCE_LEVEL, Funktion...","{'start': 1, 'end': 32, 'id': 'https://vimeo.c...",NaN,NaN,NaN
17879,Punkt und Koordinaten | Mathe by Daniel Jung_2,"Also wir haben eine x-Achse, eine y-Achse.","Funktionsarten, base, SENTENCE_LEVEL, Funktion...","{'start': 8, 'end': 34, 'id': 'https://vimeo.c...",NaN,NaN,NaN
17880,Schnittpunkt(e) bei Funktionen BASICS | Mathe ...,"Nochmal, was heißt das überhaupt ganz genau?","SENTENCE_LEVEL, base, Funktionen Übersicht, Fu...","{'start': 6, 'end': 37, 'id': 'https://vimeo.c...",NaN,NaN,NaN
